In [1]:
import schedule
import time
import requests
import datetime as dt
import sqlite3
from requests.exceptions import HTTPError, ConnectionError

# Function to create a SQLite database and weather table if it doesn't exist
def create_database():
    # Connect to the SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('weather_data.db')
    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()
    # Create the weather table with the specified columns if it doesn't exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS weather (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        city TEXT,
                        temperature REAL,
                        feels_like REAL,
                        humidity INTEGER,
                        wind_speed REAL,
                        sunrise_time TEXT,
                        sunset_time TEXT,
                        description TEXT,
                        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP)''')
    # Commit the changes to the database
    conn.commit()
    # Close the connection to the database
    conn.close()
create_database()

# Base URL for the OpenWeatherMap API
BASE_URL = "http://api.openweathermap.org/data/2.5/weather?"
# API key for authentication
API_KEY = 'ea642ff5d0165717d77c25f8bdb605da' 

districts = list(set([
    "Mumbai", "Pune", "Nagpur", "Aurangabad", "Nashik", "Amravati", "Kolhapur",
    "Thane", "Jalgaon", "Satara", "Solapur", "Bhandara", "Wardha",
    "Chandrapur", "Akola", "Washim", "Buldhana", "Hingoli",
    "Parbhani", "Jalna", "Osmanabad", "Latur", "Beed", "Yavatmal",
    "Ratnagiri", "Palghar", "Nandurbar", "Dhule", "Ahmednagar",
    "Sangli"
]))

fetch_count = 0
max_fetches = 10

def kelvin_to_celsius(kelvin):
    celsius = kelvin - 273.15
    return celsius

# Function to store weather data in the database
def store_weather_data(weather_data):
    # Connect to the SQLite database
    conn = sqlite3.connect('weather_data.db')
    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()
    # Insert the weather data into the weather table
    cursor.execute('''INSERT INTO weather (city, temperature, feels_like, humidity, wind_speed, sunrise_time, sunset_time, description)
                      VALUES (?, ?, ?, ?, ?, ?, ?, ?)''', 
                   (weather_data['city'], weather_data['temperature'], weather_data['feels_like_celsius'], 
                    weather_data['humidity'], weather_data['wind_speed'], weather_data['sunrise_time'], 
                    weather_data['sunset_time'], weather_data['description']))
    # Commit the changes to the database
    conn.commit()
    # Close the connection to the database
    conn.close()
    print(f"Storing data for {weather_data['city']}")

# Function to fetch weather data and store it in the database
def fetch_and_store_weather_data():
    global fetch_count
    if fetch_count >= max_fetches:
        return schedule.CancelJob
    for city in districts:
        # Define the API URL with the district name
        url = BASE_URL + "appid=" + API_KEY + "&q=" + city
        try:
            # Fetch weather data from the API
            response = requests.get(url)
            response.raise_for_status()  # Raise HTTPError for bad responses (4xx and 5xx)
            data = response.json()
            
            # Convert temperature from Kelvin to Celsius
            temp_kelvin = data['main']['temp']
            temp_celsius = kelvin_to_celsius(temp_kelvin)
            
            # Convert 'feels like' temperature from Kelvin to Celsius
            feels_like_kelvin = data['main']['feels_like']
            feels_like_celsius = kelvin_to_celsius(feels_like_kelvin)
            
            # Extract other weather data
            humidity = data['main']['humidity']
            description = data['weather'][0]['description']
            sunrise_time = dt.datetime.utcfromtimestamp(data['sys']['sunrise'] + data['timezone'])
            sunset_time = dt.datetime.utcfromtimestamp(data['sys']['sunset'] + data['timezone'])
            wind_speed = data['wind']['speed']
            
            # Create a dictionary with the weather data
            weather_data = {
                'city': city,
                'temperature': temp_celsius,
                'feels_like_celsius': feels_like_celsius,
                'humidity': humidity,
                'wind_speed': wind_speed,
                'sunrise_time': sunrise_time.strftime('%Y-%m-%d %H:%M:%S'),
                'sunset_time': sunset_time.strftime('%Y-%m-%d %H:%M:%S'),
                'description': description,
            }
            
            # Store the weather data in the database
            store_weather_data(weather_data)
        except (HTTPError, ConnectionError) as e:
            print(f"Failed to fetch data for {city}: {e}")
    fetch_count += 1

# Schedule the task to run every hour
schedule.every().hour.do(fetch_and_store_weather_data)

# Keep the script running
while True:
    schedule.run_pending()
    time.sleep(1)
    if fetch_count >= max_fetches:
        print("Max fetch count reached. Exiting program.")
        break


C:\Users\Sumit\AppData\Local\Temp\ipykernel_11184\1994100924.py:96: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  sunrise_time = dt.datetime.utcfromtimestamp(data['sys']['sunrise'] + data['timezone'])
C:\Users\Sumit\AppData\Local\Temp\ipykernel_11184\1994100924.py:97: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  sunset_time = dt.datetime.utcfromtimestamp(data['sys']['sunset'] + data['timezone'])


Storing data for Ahmednagar
Storing data for Parbhani
Storing data for Palghar
Storing data for Dhule
Storing data for Hingoli
Storing data for Akola
Storing data for Aurangabad
Storing data for Jalgaon
Storing data for Yavatmal
Storing data for Osmanabad
Storing data for Nandurbar
Storing data for Amravati
Storing data for Latur
Storing data for Nashik
Storing data for Sangli
Storing data for Mumbai
Storing data for Bhandara
Storing data for Ratnagiri
Storing data for Chandrapur
Storing data for Beed
Storing data for Buldhana
Storing data for Pune
Storing data for Nagpur
Storing data for Wardha
Storing data for Solapur
Storing data for Satara
Storing data for Washim
Storing data for Jalna
Storing data for Kolhapur
Storing data for Thane
Storing data for Ahmednagar
Storing data for Parbhani
Storing data for Palghar
Storing data for Dhule
Storing data for Hingoli
Storing data for Akola
Storing data for Aurangabad
Storing data for Jalgaon
Storing data for Yavatmal
Storing data for Osmana